# ChatDeepSeek


This will help you get started with DeepSeek's hosted [chat models](/docs/concepts/chat_models). For detailed documentation of all ChatDeepSeek features and configurations head to the [API reference](https://python.langchain.com/api_reference/deepseek/chat_models/langchain_deepseek.chat_models.ChatDeepSeek.html).

:::tip

DeepSeek's models are open source and can be run locally (e.g. in [Ollama](./ollama.ipynb)) or on other inference providers (e.g. [Fireworks](./fireworks.ipynb), [Together](./together.ipynb)) as well.

:::

## Overview
### Integration details

| Class | Package | Local | Serializable | [JS support](https://js.langchain.com/docs/integrations/chat/deepseek) | Package downloads | Package latest |
| :--- | :--- | :---: | :---: |  :---: | :---: | :---: |
| [ChatDeepSeek](https://python.langchain.com/api_reference/deepseek/chat_models/langchain_deepseek.chat_models.ChatDeepSeek.html) | [langchain-deepseek](https://python.langchain.com/api_reference/deepseek/) | ❌ | beta | ✅ | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-deepseek?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-deepseek?style=flat-square&label=%20) |

### Model features
| [Tool calling](/docs/how_to/tool_calling) | [Structured output](/docs/how_to/structured_output/) | JSON mode | [Image input](/docs/how_to/multimodal_inputs/) | Audio input | Video input | [Token-level streaming](/docs/how_to/chat_streaming/) | Native async | [Token usage](/docs/how_to/chat_token_usage_tracking/) | [Logprobs](/docs/how_to/logprobs/) |
| :---: | :---: | :---: | :---: |  :---: | :---: | :---: | :---: | :---: | :---: |
| ✅ | ✅ | ❌ | ❌ | ❌ | ❌ | ✅ | ✅ | ✅ | ❌ |

:::note

DeepSeek-R1, specified via `model="deepseek-reasoner"`, does not support tool calling or structured output. Those features [are supported](https://api-docs.deepseek.com/guides/function_calling) by DeepSeek-V3 (specified via `model="deepseek-chat"`).

:::

## Setup

To access DeepSeek models you'll need to create a/an DeepSeek account, get an API key, and install the `langchain-deepseek` integration package.

### Credentials

Head to [DeepSeek's API Key page](https://platform.deepseek.com/api_keys) to sign up to DeepSeek and generate an API key. Once you've done this set the `DEEPSEEK_API_KEY` environment variable:

In [8]:
import getpass
import os

if not os.getenv("DEEPSEEK_API_KEY"):
    os.environ["DEEPSEEK_API_KEY"] = 'sk-e157ca06a73547a390f2b3794fbf805d'

To enable automated tracing of your model calls, set your [LangSmith](https://docs.smith.langchain.com/) API key:

In [ ]:
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")

### Installation

The LangChain DeepSeek integration lives in the `langchain-deepseek` package:

In [2]:
%pip install -qU langchain-deepseek

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.0 MB/s eta 0:00:00


## Instantiation

Now we can instantiate our model object and generate chat completions:

In [11]:
from langchain_deepseek import ChatDeepSeek

llm = ChatDeepSeek(
    model="deepseek-reasoner",
    api_key='sk-e157ca06a73547a390f2b3794fbf805d',
    # other params...
)

## Invocation

In [6]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='La traduction de "I love programming" en français est :\n\n**J\'aime la programmation.**\n\n### Explications :\n- **J\'aime** : Traduction de "I love" (verbe *aimer* conjugué à la première personne).\n- **la programmation** : Correspond à "programming" dans le contexte informatique. En français, ce terme désigne spécifiquement l\'activité de création de programmes informatiques.\n\n### Alternative courante :\nSi le contexte est informel ou général (ex. plaisir de coder), on peut aussi dire :  \n**J\'adore coder.** (*Coder* étant un terme familier pour *programmer*).', additional_kwargs={'refusal': None, 'reasoning_content': 'We are translating from English to French.\n The sentence is: "I love programming."\n In French:\n   "I" -> "Je"\n   "love" -> "aime" (first person singular of the verb "aimer")\n   "programming" -> "la programmation" (as in the activity of writing computer programs)\n\n So the translation is: "J\'aime la programmation."\n\n Note: In French, we u

## Chaining

We can [chain](/docs/how_to/sequence/) our model with a prompt template like so:

In [22]:
prompt ="""
You are an expert at breaking down search queries into focused sub-queries to improve retrieval.
Your task is to decompose the following query into 2-4 focused sub-queries that will help retrieve
comprehensive information to answer the original query.

Original query: what is ai

For any query, create sub-queries that:
1. Focus on different aspects or components of the original query
2. Are self-contained and answerable independently
3. Use specific terminology and keywords for better retrieval
4. Cover the breadth of information needed to answer the original query

Even for simple queries, create variations that:
- Use different keywords or synonyms
- Focus on specific aspects of the topic
- Ask for different types of information (definitions, examples, explanations, etc.)

Return your sub-queries as a JSON list with the following format:
{{
  "sub_queries": [
    "First focused sub-query here",
    "Second focused sub-query here",
    "Third focused sub-query here"
  ]}}
"""
response_schema = {
  "title": "QueryBreakdown",
  "description": "Breakdown of query into focused sub-queries",
  "type": "object",
  "properties": {
      "sub_queries": {
          "type": "array",
          "items": {"type": "string"},
          "description": "List of focused sub-queries to search for"
      }
  },
  "required": ["sub_queries", "reasoning"]
}
structured_llm = llm.with_structured_output(response_schema,method='json_mode')
# Add instructions to the prompt to return the output in JSON format
structured_llm.invoke(prompt)

{'sub_queries': ['Define artificial intelligence and its core principles',
  'Provide examples of artificial intelligence applications in daily life',
  'Explain the key technologies in artificial intelligence such as machine learning and neural networks']}

In [6]:
from pydantic import BaseModel, Field

class GetWeather(BaseModel):
    '''Get the current weather in a given location'''

    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")

class GetPopulation(BaseModel):
    '''Get the current population in a given location'''

    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")

llm_with_tools = llm.bind_tools([GetWeather, GetPopulation])
ai_msg = llm_with_tools.invoke("Which city is hotter today and which is bigger: LA or NY?")
ai_msg.tool_calls

[{'name': 'GetWeather',
  'args': {'location': 'Los Angeles, CA'},
  'id': 'call_0_c10a54e2-00a6-4d12-824d-5ee3eb0449d9',
  'type': 'tool_call'},
 {'name': 'GetWeather',
  'args': {'location': 'New York, NY'},
  'id': 'call_1_da99a803-65cf-4bac-a831-c7d549460e8c',
  'type': 'tool_call'}]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

## API reference

For detailed documentation of all ChatDeepSeek features and configurations head to the [API Reference](https://python.langchain.com/api_reference/deepseek/chat_models/langchain_deepseek.chat_models.ChatDeepSeek.html).